# This Project is about extracting  teams' data from [Elite Prospects hockey](https://www.eliteprospects.com/teams) Website

### Data required from each team page is to get every player following information
- Player Team
- Jersey
- Position
- Name
- Born year
- Weight
- hight
- S
- Birthplace
- => last information need to get is every player last team he played for which you have to find it form his page infromation

### I have run this code for three different teams. but we can run it as many as we want

In [1]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
teams_urls_list = [
                  'https://www.eliteprospects.com/team/6722/adrian-college#players',
                  'https://www.eliteprospects.com/team/2/brynas-if',
                  'https://www.eliteprospects.com/team/1446/cholet'
                  ]

In [21]:
page = requests.get(teams_urls_list[2])

soup = BeautifulSoup(page.content, 'html.parser')

In [22]:
all_team_data = []

In [23]:
rostertable = soup.find('table', {'data-sort-ajax-container':'#roster'})
rows = rostertable.find_all('tr')

In [24]:
# teamname = soup.find('div', id='name-and-logo').find('h1', 'semi-logo').text.strip()
# remove = soup.find('div', id='name-and-logo').find('h1', 'semi-logo').a.text.strip()
# teamname = teamname.strip(remove).strip()
# teamname

In [25]:
position = None

def changeposition(n):
    global position
    position = rows[n].td.text.strip()[0]

In [26]:
for i in range(1,len(rows)):
    
    tds = rows[i].find_all('td')
    
    if len(tds) == 1 :
        changeposition(i)
        continue    
    
    try: 
        jersey = rows[i].find('td', 'jersey').text.strip() 
    except: 
        jersey = None
        
    try: 
        Player = rows[i].find('td', 'sorted').span.a.text.strip()
    except: 
        Player = None
        
    try: 
        Born = rows[i].find('td', 'date-of-birth').text.strip() 
    except: 
        Born = None
        
    try: 
        WT = rows[i].find('td', 'weight').text.strip()
    except:
        WT = None
        
    try: 
        HT = rows[i].find('td', 'height').text.strip() 
    except: 
        HT = None
        
    try: 
        S = rows[i].find('td', 'shoots').text.strip() 
    except: S = None
        
    try: 
        Birthplace = rows[i].find('td', 'birthplace').text.strip() 
    except: 
        Birthplace = None
        
    # get the last team this player have played for
    lastname = None
    playerlink = rows[i].find('td', 'sorted').a.get('href')
    playerpage = requests.get(playerlink)
    playersoup = BeautifulSoup(playerpage.content, 'html.parser')
    teamname = playersoup.find('div', "ep-entity-header__text").a.text.strip()
    teams = playersoup.find('div', id='league-stats').find_all('td', 'team')
    teamnames = []
    for team in teams:
        try:
            teamnames.append(team.a.text)
        except:
            pass
    lastclub = None
    xx = [club.strip() for club in list(dict.fromkeys(reversed(teamnames))).copy()]
    try:
        while xx[0] != teamname:
            xx.pop(0)
        if len(xx)> 2 : lastname = xx[1]
    except: pass
    
    all_team_data.append([teamname, jersey, position, Player, Born,
                          WT, HT, S, Birthplace, lastname])


In [27]:
columns = ['Team', '#', 'playerposition', 'Player', 'Born', 'WT', 'HT', 'S', 'Birthplace', 'lastteam']

In [28]:
pd.set_option('display.max_rows', 100)
df = pd.DataFrame(data=all_team_data , columns=columns)

# First Team Test <a class="anchor" id="1"></a>
## [See team page](https://www.eliteprospects.com/team/6722/adrian-college#players)

In [11]:
df

,Team,#,playerposition,Player,Born,WT,HT,S,Birthplace,lastteam
0,Adrian College,#1,G,Noah Decottignies (G),2000,82,185,L,"Calgary, AB, CAN",Columbia Valley Rockies
1,Adrian College,#35,G,Harrison Fleming (G),2000,91,193,R,"Brighton, MI, USA",Dundas Blues
2,Adrian College,#30,G,Dylan Knighton (G),2000,82,185,L,"Pequannock, NJ, USA",Protec Jr Ducks
3,Adrian College,#4,D,Dakota Bohn (D),1998,91,191,L,"Saginaw, MI, USA",Chatham Maroons
4,Adrian College,#25,D,Matthew Cruz (D),2004,82,183,-,"New Baltimore, MI, USA",None
5,Adrian College,#5,D,Dylan Kent (D),1998,98,191,L,"Beaumont, AB, CAN",Kelowna Chiefs
6,Adrian College,#77,D,Ethan McLaughlin (D),2000,79,180,-,"Penticton, BC, CAN",Nelson Leafs
7,Adrian College,#2,D,Sheldon Nolan (D),1998,86,188,L,"Richmond Hill, ON, CAN",Newmarket Hurricanes
8,Adrian College,#9,D,Brodie Thornton (D/F),1999,82,185,L,"Macomb Township, MI, USA",Metro Jets
9,Adrian College,#6,D,Mason Vaughan (D),1998,84,188,L,"Waterford, MI, USA",Idaho IceCats


# Second Team Test <a class="anchor" id="2"></a>
## [See team page](https://www.eliteprospects.com/team/2/brynas-if)

In [20]:
df

,Team,#,playerposition,Player,Born,WT,HT,S,Birthplace,lastteam
0,Hudiksvalls HC,#30,G,Andreas Bosson (G),1999,89,198,L,None,Alfta GIF
1,Brynäs IF,#45,G,Kristers Gudlevskis (G),1992,97,192,L,None,HC Slovan Bratislava
2,Brynäs IF,#35,G,Veini Vehviläinen (G),1997,82,183,L,None,Toronto Marlies
3,Brynäs IF,#2,D,Kristofer Berglund (D),1988,82,177,L,None,Dinamo Riga
4,Brynäs IF,#63,D,Julius Bergman (D),1995,93,186,R,None,Pelicans
5,Brynäs IF,#15,D,Simon Bertilsson (D),1991,84,184,L,None,Sweden (all)
6,Brynäs IF,#7,D,Marcus Björk (D),1997,92,191,R,None,Malmö Redhawks
7,Brynäs IF,#38,D,Oskars Cibulskis (D),1988,86,188,L,None,Dinamo Riga
8,Brynäs IF,#5,D,Chay Genoway (D),1986,73,176,L,None,Avtomobilist Yekaterinburg
9,Brynäs IF,#62,D,Tom Hedberg (D),1999,81,181,L,None,MODO Hockey


# Third Team Test <a class="anchor" id="3"></a>
## [See team page](https://www.eliteprospects.com/team/1446/cholet)

In [29]:
df

,Team,#,playerposition,Player,Born,WT,HT,S,Birthplace,lastteam
0,Angers II,#20,G,Victor Bodin (G),2001,87,185,L,"Singapore, SGP",Vaujany
1,Angers,,G,Elliot Leveque (G),2003,57,175,L,"Cholet, FRA",Cholet
2,Cholet,#1,G,Michael Luba (G),1995,78,180,L,"Montréal, QC, CAN",Chambéry
3,Angers II,#90,G,Mathys Schwerdel (G),2001,60,167,R,"Echirolles, FRA",Angers U20
4,Angers II,#9,D,Léonard Baril (D),2002,72,170,L,"St-Sébastien-sur-Loire, FRA",Angers U20
5,Cholet,#77,D,Martin Borovsky (D),1988,84,180,R,"Spisska Nova Ves, SVK",HK Spisska Nova Ves
6,Cholet,#33,D,Brieuc Houeix (D),1997,75,179,L,"Angers, FRA",Nantes
7,Cholet,#55,D,Ludovic Karsh (D),1998,80,183,L,"Drummondville, QC, CAN",Longueuil Collège-Français
8,Cholet,#7,D,Drayson Pears (D),1998,86,183,R,"Victoria, BC, CAN",Univ. of Alaska-Anchorage
9,Cholet,#65,D,Joris Rama (D),1999,72,180,L,"Gap, FRA",Gap II
